In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
folder_paths = ['/content/drive/MyDrive/datst/Boat_person', '/content/drive/MyDrive/datst/None', '/content/drive/MyDrive/datst/all',
                '/content/drive/MyDrive/datst/lake_person', '/content/drive/MyDrive/datst/only_boat', '/content/drive/MyDrive/datst/only_fish',
                '/content/drive/MyDrive/datst/only_lake', '/content/drive/MyDrive/datst/only_person', '/content/drive/MyDrive/datst/person_fish']
# Define the transformation to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Rescale the images to 224x224
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Create a custom dataset class
class ImageFolderDataset(Dataset):
    def __init__(self, folder_paths, transform=None):
        self.samples = []
        self.targets = []
        
        for i, folder_path in enumerate(folder_paths):
            for root, _, filenames in os.walk(folder_path):
                for filename in filenames:
                    if not filename.startswith('.'):
                        self.samples.append(os.path.join(root, filename))
                        self.targets.append(i)
        
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.samples[index]
        label = self.targets[index]
        
        image = Image.open(image_path).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label
    
    def __len__(self):
        return len(self.samples)

# Create the dataset
dataset = ImageFolderDataset(folder_paths, transform)

# Create the data loader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, len(folder_paths))  # Modify the last fully connected layer to match the number of classes

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the classifier
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")

# Save the trained model
torch.save(model.state_dict(), 'classifier_model.pth')